# Homework 4

Use this resource as a reference for implementing the algorithm: https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-messages

The following may also be helpful:

https://www.rareskills.io/post/finite-fields

https://www.rareskills.io/post/elliptic-curve-addition

https://www.rareskills.io/post/elliptic-curves-finite-fields

https://rareskills.io/post/ecdsa-tutorial

Implement ECDSA from scratch. You want to use the secp256k1 curve (which specifies the values for the curve). When starting off, use the Elliptic curve multiplication library used in the blog post linked here: https://www.rareskills.io/post/generate-ethereum-address-from-private-key-python

1) pick a private key

2) generate the public key using that private key (not the eth address, the public key)

3) pick message m and hash it to produce h (h can be though of as a 256 bit number)

4) sign m using your private key and a randomly chosen nonce k. produce (r, s, h, PubKey)

5) verify (r, s, h, PubKey) is valid

You may use a library for point multiplication, but everything else you must do from scratch. Remember, when you compute the multiplicative inverse, you need to do it with respect to the curve order.

Pay close attention to the distinction between the curve order and the prime number $p$ we compute the modulus of $y^2=x^3+b \pmod p$.

In [49]:
# Picking a private key
coinFlip = b'1100001011001111110001001100101000001111101101110011001000110001101100011101101011010001011000101110101000010101001001101110111011001000100001000111111100001100100110010010111110110100000010011111100000101011001110000101100101011111001101010001100001000'


In [50]:
# 2 means base 2 for binary

private_key = hex(int(coinFlip, 2))
private_key

'0x1859f89941f6e646363b5a2c5d42a4ddd9108fe19325f6813f05670b2be6a308'

In [51]:
# Install ecpy & sha3
!pip install ecpy

In [52]:
# Import Libraries
from ecpy.curves import Curve
from ecpy.keys import ECPublicKey, ECPrivateKey

In [53]:
# Using the private key from above
private_key = 0x1859f89941f6e646363b5a2c5d42a4ddd9108fe19325f6813f05670b2be6a308

In [54]:
# Load the curve and get public key
cv = Curve.get_curve('secp256k1')
pv_key = ECPrivateKey(private_key, cv)
pu_key = pv_key.get_public_key()

print('public key: ', pu_key)

public key:  ECPublicKey:
  x: 12c089b597340faceb61a728fb8ef87f1f85b466d295091a16344cfdda249344
  y: 2195c27ecddf06c2a9aef26b235ae088393556ab0eff9b3384fd3a82d65a9335


In [55]:
# import hashlib
import hashlib

In [56]:
# Your message
message = "Hello, ECDSA!"

In [57]:
# Hash the message
# The message needs to be encoded to bytes first
message_bytes = message.encode('utf-8')
hash_object = hashlib.sha256(message_bytes)

In [58]:
# Get the hash as a hexadecimal string
h_hex = hash_object.hexdigest()

# Convert to an integer (this is what you'll use in ECDSA calculations)
h = int(h_hex, 16)

print(f"Message: {message}")
print(f"Hash (hex): {h_hex}")
print(f"Hash (int): {h}")

Message: Hello, ECDSA!
Hash (hex): 6877498347a58bf169c716d157a503ca85f5f68720d7986a4bd6a9217ad896ca
Hash (int): 47251298420149935885156812592629867927474418964758231288704275995193241016010


In [59]:
G = cv.generator

In [60]:
pv_key_pt = private_key * G
pv_key_pt

In [61]:
# Asserting, P = pG, where P is public key and p is private key
# print(f"Elliptic curve discreet log pv_x: {pv_key_pt.x}")
# print(f"Elliptic curve discreet log pv_y: {pv_key_pt.y}")

# print(f"Public key, x: {pu_key.W.x}")
# print(f"Public key, y: {pu_key.W.y}")
assert pv_key_pt.x == pu_key.W.x
assert pv_key_pt.y == pu_key.W.y

In [62]:
import secrets

# secp256k1 curve order
n = cv.order

# Generate random nonce k in range [1, n-1]
k = secrets.randbelow(n - 1) + 1

print(f"Random nonce k: {k}")
print(f"In hex: {hex(k)}")

Random nonce k: 47392526271311164485372901260433979253757923366588055908087956914652228766333
In hex: 0x68c738236befaa707142bbf7adeb1e4b485caa246874631aa8c16d65dff24a7d


In [63]:
# Based on formula: https://rareskills.io/post/ecdsa-tutorial#the-complete-ecdsa-algorithm-step-by-step
R = k * G # the Random EC point
r = R.x # only use the x component
k_inv = pow(k, -1, n) # calculate the multiplicative inverse of k
s = (k_inv * (h + r * private_key)) % n
print(f"Signature: {s,r}")

Signature: (94378775072824458048237043408803440608936168265413237722176879707199180560305, 110290884772806598751103419975488585107313231230163720430456935138526254243222)


In [64]:
# Calculating R' for verification
# Reference formula: R' = s_inv(hG + rP)

s_inv = pow(s, -1, n)
R_p = s_inv * (h * G + r * pu_key.W) # Apparently we cannot do this, we need to do use the way below
print(f"R', x: {R_p.x}, y: {R_p.y}")
print(f"R, x: {R.x}, y: {R.y}")

R', x: 110290884772806598751103419975488585107313231230163720430456935138526254243222, y: 43745958832785847252894490171786164593457657872485935153337637302510357401869
R, x: 110290884772806598751103419975488585107313231230163720430456935138526254243222, y: 43745958832785847252894490171786164593457657872485935153337637302510357401869


In [65]:
# Verification
s_inv = pow(s, -1, n)

# Compute u1 and u2 (these are scalars)
u1 = (h * s_inv) % n
u2 = (r * s_inv) % n

# Compute the point R' = u1*G + u2*PubKey
# Do the scalar multiplications first, then add the points
R_p = u1 * G + u2 * pu_key.W

print(f"R', x: {R_p.x}, y: {R_p.y}")
print(f"R, x: {R.x}, y: {R.y}")

# Verification check
if R_p.x % n == r:
    print("Signature is VALID!")
else:
    print("Signature is INVALID!")

R', x: 110290884772806598751103419975488585107313231230163720430456935138526254243222, y: 43745958832785847252894490171786164593457657872485935153337637302510357401869
R, x: 110290884772806598751103419975488585107313231230163720430456935138526254243222, y: 43745958832785847252894490171786164593457657872485935153337637302510357401869
Signature is VALID!
